In [6]:
# Install nbformat if missing (required for plotly rendering in some environments)
!pip install nbformat --upgrade

In [1]:
import sys
import importlib

# Force re-check of nbformat
try:
    import nbformat
    print(f"nbformat version: {nbformat.__version__}")
except ImportError:
    print("nbformat still not importable")

# Reload plotly modules to pick up nbformat
import plotly.io._renderers
importlib.reload(plotly.io._renderers)
print("Reloaded plotly renderers")

nbformat version: 5.10.4
Reloaded plotly renderers


In [2]:
import plotly.io
importlib.reload(plotly.io)
print("Reloaded plotly.io")

Reloaded plotly.io


# Troubleshooting CPT Plotting for ID 314

This notebook isolates the plotting logic for CPT data (QC and Rf) to debug visibility issues. We will focus on `sondering_id = 314`.

In [3]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

In [4]:
# Load the raw data
data_path = '../data/test_raw_data.csv'
df = pd.read_csv(data_path)
print(f"Loaded data with shape: {df.shape}")
df.head()

Loaded data with shape: (81992, 19)


/tmp/ipykernel_77313/3823267650.py:3: DtypeWarning:

Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.



,sondering_id,index,pkey_sondering,sondeernummer,x,y,start_sondering_mtaw,diepte_sondering_tot,diepte,diepte_mtaw,qc,fs,qtn,rf,fr,icn,sbt,ksbt,lithostrat_id
0,314,2593,https://www.dov.vlaanderen.be/data/sondering/1...,GEO-97/127-S2,153278.2,181734.6,15.26,25.4,1.6,13.66,1.17,0.035,35.894004,2.991453,3.058371,2.564340,5.0,1.434000e-07,Quartair
1,314,2594,https://www.dov.vlaanderen.be/data/sondering/1...,GEO-97/127-S2,153278.2,181734.6,15.26,25.4,1.7,13.56,1.57,0.033,42.562319,2.101911,2.138968,2.406724,5.0,4.321000e-07,Quartair
2,314,2595,https://www.dov.vlaanderen.be/data/sondering/1...,GEO-97/127-S2,153278.2,181734.6,15.26,25.4,1.8,13.46,1.43,0.036,38.536991,2.517483,2.569226,2.491219,5.0,2.392000e-07,Quartair
3,314,2596,https://www.dov.vlaanderen.be/data/sondering/1...,GEO-97/127-S2,153278.2,181734.6,15.26,25.4,1.9,13.36,0.50,0.024,15.678501,4.800000,5.111166,2.982185,3.0,7.700000e-09,Quartair
4,314,2597,https://www.dov.vlaanderen.be/data/sondering/1...,GEO-97/127-S2,153278.2,181734.6,15.26,25.4,2.0,13.26,1.33,0.023,33.203119,1.729323,1.772110,2.440158,5.0,3.419000e-07,Quartair


In [5]:
# Filter for ID 314
sondering_id = 314
# Ensure ID is string for consistency with app logic, or int if csv has int
# The CSV preview showed 314 as int/string. Let's check dtypes.
print(f"Sondering ID dtype: {df['sondering_id'].dtype}")

# Filter
# Handle both int and string just in case
if df['sondering_id'].dtype == 'O':
    cpt_raw = df[df['sondering_id'].astype(str) == str(sondering_id)].copy()
else:
    cpt_raw = df[df['sondering_id'] == sondering_id].copy()

print(f"Rows for ID {sondering_id}: {len(cpt_raw)}")

# Clean numeric columns (handling commas if any, though CSV looked like dots)
def clean_numeric(series):
    return pd.to_numeric(series.astype(str).str.replace(',', '.'), errors='coerce')

cpt_raw['diepte'] = clean_numeric(cpt_raw['diepte'])
cpt_raw['qc'] = clean_numeric(cpt_raw['qc'])
cpt_raw['rf'] = clean_numeric(cpt_raw['rf'])

# Drop NaNs
cpt_raw = cpt_raw.dropna(subset=['qc', 'rf', 'diepte'])
print(f"Rows after cleaning: {len(cpt_raw)}")

cpt_raw = cpt_raw.sort_values('diepte')
cpt_raw[['diepte', 'qc', 'rf']].head()

Sondering ID dtype: int64
Rows for ID 314: 239
Rows after cleaning: 239


,diepte,qc,rf
0,1.6,1.17,2.991453
1,1.7,1.57,2.101911
2,1.8,1.43,2.517483
3,1.9,0.50,4.800000
4,2.0,1.33,1.729323


In [6]:
# Replicating the logic from app.py
fig = go.Figure()

# Layout setup
layout_axes = {}
start = 0
end = 1.0 # Full width for single plot

axis_idx = 1
base_axis_name = 'xaxis'
base_axis_ref = 'x'

rf_idx = 2 # unique axis id for the Rf overlay
rf_axis_name = f'xaxis{rf_idx}'
rf_axis_ref = f'x{rf_idx}'

y_axis_name = 'yaxis'
y_axis_ref = 'y'

# QC trace
fig.add_trace(go.Scatter(
    x=cpt_raw['qc'], y=cpt_raw['diepte'], mode='lines',
    name='QC', line=dict(color='black', width=1.6),
    xaxis=base_axis_ref, yaxis=y_axis_ref
))

# Rf trace (top axis)
fig.add_trace(go.Scatter(
    x=cpt_raw['rf'], y=cpt_raw['diepte'], mode='lines',
    name='Rf', line=dict(color='purple', width=1.4),
    xaxis=rf_axis_ref, yaxis=y_axis_ref
))

# Axis layouts
qc_min = cpt_raw['qc'].min()
qc_max = cpt_raw['qc'].max()
rf_min = cpt_raw['rf'].min()
rf_max = cpt_raw['rf'].max()

# Add 5% padding
qc_range = [qc_min - (qc_max-qc_min)*0.05, qc_max + (qc_max-qc_min)*0.05]
rf_range = [rf_min - (rf_max-rf_min)*0.05, rf_max + (rf_max-rf_min)*0.05]

layout_axes[base_axis_name] = dict(
    domain=[start, end], 
    anchor=y_axis_ref, 
    title='QC (MPa)',
    range=qc_range
)
layout_axes[rf_axis_name] = dict(
    domain=[start, end], 
    anchor=y_axis_ref,
    overlaying=base_axis_ref, 
    side='top',
    title=dict(text='Rf (%)', font=dict(color='purple')), 
    tickfont=dict(color='purple'),
    range=rf_range
)
layout_axes[y_axis_name] = dict(
    domain=[0, 1], 
    title='Depth (m)', 
    autorange='reversed', 
    anchor=base_axis_ref
)

fig.update_layout(**layout_axes)
fig.update_layout(title=f'CPT {sondering_id}: QC and Rf Profiles', showlegend=True, height=800)

fig.show()

> **Important:** If you encounter a `ValueError: Mime type rendering requires nbformat...` error below, please **restart the kernel** and run the cells again. The `nbformat` library was installed in the previous steps but requires a kernel restart to be fully recognized by Plotly.

In [ ]:
# Alternative approach: Side-by-Side to verify data visibility
# If this works, the data is fine and the issue is the overlay/domain logic.

fig_side = make_subplots(rows=1, cols=2, shared_yaxes=True, subplot_titles=("QC", "Rf"))

fig_side.add_trace(go.Scatter(x=cpt_raw['qc'], y=cpt_raw['diepte'], name='QC', line=dict(color='black')), row=1, col=1)
fig_side.add_trace(go.Scatter(x=cpt_raw['rf'], y=cpt_raw['diepte'], name='Rf', line=dict(color='purple')), row=1, col=2)

fig_side.update_yaxes(autorange='reversed', title='Depth (m)')
fig_side.update_xaxes(title='QC (MPa)', row=1, col=1)
fig_side.update_xaxes(title='Rf (%)', row=1, col=2)

fig_side.update_layout(title='Side-by-Side Check', height=600)
fig_side.show()